晓得我写的辣鸡啦

当时时间紧（就最后几分钟了，所以由一段程序跑了一半就没有跑了）

其他剩下的都是按照程序步骤来的

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
# 没有导入交叉验证的包

In [2]:
path_train = 'data_file/train.csv'
path_test = 'data_file/test.csv'

data_train = pd.read_csv(path_train, header=0)
data_test = pd.read_csv(path_test, header=0)

In [3]:
data_train.head(5)

,XINGBIE,AGE,HYZK,ZHIYE,ZHICHEN,ZHIWU,XUELI,DWJJLX,DWSSHY,GRJCJS,GRZHZT,GRZHYE,GRZHSNJZYE,GRZHDNGJYE,GRYJCE,DKFFE,DKYE,DKLL,label
0,1,18,90,90,999,0,99,150,12,1737.0,1,3223.515,801.310,837.000,312.00,175237,154112.935,2.708,0
1,2,34,90,90,999,0,99,110,0,4894.0,1,18055.195,53213.220,1065.200,795.84,300237,298252.945,2.979,0
2,1,27,90,90,999,0,99,150,9,10297.0,1,27426.600,13963.140,7230.020,1444.20,150237,147339.130,2.708,0
3,1,37,90,90,999,0,99,150,7,10071.5,1,111871.130,99701.265,2271.295,1417.14,350237,300653.780,2.708,0
4,2,33,90,90,999,0,99,900,14,2007.0,1,237.000,11028.875,35.780,325.50,150237,145185.010,2.708,0


In [4]:
data_train['label'].value_counts()

0    37243
1     2757
Name: label, dtype: int64

In [5]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 19 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   XINGBIE     40000 non-null  int64  
 1   AGE         40000 non-null  int64  
 2   HYZK        40000 non-null  int64  
 3   ZHIYE       40000 non-null  int64  
 4   ZHICHEN     40000 non-null  int64  
 5   ZHIWU       40000 non-null  int64  
 6   XUELI       40000 non-null  int64  
 7   DWJJLX      40000 non-null  int64  
 8   DWSSHY      40000 non-null  int64  
 9   GRJCJS      40000 non-null  float64
 10  GRZHZT      40000 non-null  int64  
 11  GRZHYE      40000 non-null  float64
 12  GRZHSNJZYE  40000 non-null  float64
 13  GRZHDNGJYE  40000 non-null  float64
 14  GRYJCE      40000 non-null  float64
 15  DKFFE       40000 non-null  int64  
 16  DKYE        40000 non-null  float64
 17  DKLL        40000 non-null  float64
 18  label       40000 non-null  int64  
dtypes: float64(7), int64(12)


In [12]:
x, y = data_train.iloc[:, :-1], data_train.iloc[:, -1]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 0)

# 继承类，生成随机森林分类器的子类
# TODO: 调参
# forest = RandomForestClassifier(n_estimators=1000, random_state=0, n_jobs=-1)
forest = RandomForestClassifier(oob_score=True, random_state=0)
forest.fit(x_train, y_train)
# param_dict = {"n_estimators": [100,120,200,300,500,800], "max_depth": [3,5,8,10,15], "max_features":["auto","log2"]}
# estimator = GridSearchCV(estimator, param_grid=param_dict, cv=3)

RandomForestClassifier(oob_score=True, random_state=0)

In [13]:
forest.oob_score_

0.95675

In [15]:
# feat_labels = df.columns[1:23]
feat_labels = data_train.columns[:-1]
importances = forest.feature_importances_ # 特征的重要性
indices = np.argsort(importances)[::-1]
for f in range(x_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30, feat_labels[indices[f]], importances[indices[f]]))

 1) DWJJLX                         0.135636
 2) DWSSHY                         0.114818
 3) GRYJCE                         0.110192
 4) GRJCJS                         0.097681
 5) GRZHDNGJYE                     0.091564
 6) DKYE                           0.085398
 7) GRZHSNJZYE                     0.081277
 8) GRZHYE                         0.077251
 9) AGE                            0.064366
10) DKFFE                          0.060897
11) GRZHZT                         0.054012
12) DKLL                           0.012891
13) XINGBIE                        0.012365
14) ZHICHEN                        0.001056
15) ZHIYE                          0.000571
16) HYZK                           0.000008
17) XUELI                          0.000008
18) ZHIWU                          0.000008


In [22]:
from bayes_opt import  BayesianOptimization
from sklearn.model_selection import cross_val_score

In [23]:
x, y = data_train.iloc[:, :-1], data_train.iloc[:, -1]

In [24]:
rf = RandomForestClassifier()
print(np.mean(cross_val_score(rf, x, y, cv=5, scoring='roc_auc')))

0.9153152675563195


In [33]:
def rf_cv(n_estimators, min_samples_split, min_samples_leaf, max_features, max_depth):
    val = cross_val_score(
        RandomForestClassifier(n_estimators=int(n_estimators),
            min_samples_split=int(min_samples_split),
            min_samples_leaf=int(min_samples_leaf),
            max_features=min(max_features, 0.999), # float
            max_depth=int(max_depth),
            random_state=0
        ),
        x, y, scoring='roc_auc', cv=5
    ).mean()
    return val

In [34]:
 rf_bo = BayesianOptimization(
        rf_cv,
        {'n_estimators': (10, 250),
        'min_samples_split': (2, 25),
        'min_samples_leaf': (10, 61),
        'max_features': (0.1, 0.5),
        'max_depth': (5, 15)}
    )

In [35]:
rf_bo.maximize()

|   iter    |  target   | max_depth | max_fe... | min_sa... | min_sa... | n_esti... |
-------------------------------------------------------------------------------------
|  1        |  0.8861   |  5.125    |  0.3481   |  33.81    |  15.42    |  228.7    |
|  2        |  0.9166   |  11.38    |  0.4868   |  27.34    |  22.46    |  163.1    |
|  3        |  0.9036   |  8.401    |  0.3233   |  58.37    |  13.67    |  87.03    |
|  4        |  0.8923   |  6.88     |  0.282    |  28.49    |  18.83    |  53.85    |
|  5        |  0.9124   |  14.78    |  0.4435   |  59.62    |  6.594    |  137.5    |
|  6        |  0.9097   |  13.59    |  0.3062   |  59.69    |  5.991    |  140.2    |
|  7        |  0.9163   |  15.0     |  0.5      |  34.72    |  20.75    |  144.7    |
|  8        |  0.9134   |  15.0     |  0.5      |  51.07    |  25.0     |  122.0    |
|  9        |  0.9208   |  15.0     |  0.5      |  10.0     |  6.395    |  146.8    |
|  10       |  0.8859   |  5.0      |  0.5      |  10.

KeyboardInterrupt: 

In [37]:
rf_final = RandomForestClassifier(min_samples_split=6, min_samples_leaf=10, max_depth=15, max_features=0.5, random_state=10, n_jobs=-1)
rf_final.fit(x, y)

RandomForestClassifier(max_depth=15, max_features=0.5, min_samples_leaf=10,
                       min_samples_split=6, n_jobs=-1, random_state=10)

In [38]:
y_predict = rf_final.predict(data_test)

In [18]:
gsearch_1.best_params_, gsearch_1.best_score_

({'n_estimators': 1000}, 0.9017391906772112)

In [10]:
x_test = pd.read_csv(path_test)
y_predict = forest.predict(x_test)
print("y_predict:\n", y_predict)

y_predict:
 [0 0 0 ... 0 0 0]
1066


In [11]:
print(np.sum(y_predict)/len(y_predict))

0.07106666666666667


In [41]:
d = pd.DataFrame()
d['id'] = pd.read_csv('data_file/submit.csv')['id']
d['label'] = y_predict
d.to_csv('data_file/submit_randomforest.csv',encoding='utf-8', index=0)